In [ ]:
#两种情况 
#1.数据量大，内存不够。
#2.用于在线学习，增量训练。
batch_size = 1000
num_batch = train.shape[0] // batch_size

auc_score = []
for batch in range(num_batch):
    start = batch * batch_size
    end = start + batch_size
    x = train[start: end]
    y = y_train[start: end]

    train_df = lgb.Dataset(x, y,)
    val_df = lgb.Dataset(test, y_test)
    if start == 0:
        init_model = None
    else:
        init_model = clf
    clf = lgb.train(params_lgb, # 使用贝叶斯调参最优参数
    train_df, num_boost_round=20,
    valid_sets = [val_df], verbose_eval=100, 
    init_model=init_model,
    early_stopping_rounds=50,
    keep_training_booster=True)

    val_pre = clf.predict(test, num_iteration=clf.best_iteration)
    auc_score.append(roc_auc_score(y_test, val_pre))

print(auc_score)  
print('测试集auc均值',np.mean(auc_score))

In [ ]:
#学习率衰减符合最优点的寻找逻辑，先快后慢
lgb_train = lgb.Dataset(train, y_train)
lgb_val = lgb.Dataset(test, y_test)

# 指数衰减
clf = lgb.train(
    params_lgb, # 使用贝叶斯调参最优参数
    lgb_train, 
    num_boost_round=20,
    valid_sets = [lgb_val], 
    verbose_eval=5, 
    learning_rates=lambda iter: 0.05 * (0.99 ** iter),
    early_stopping_rounds=5)
val_pre = clf.predict(test, num_iteration=clf.best_iteration)
print(roc_auc_score(y_test, val_pre))

In [ ]:
# 阶梯衰减
clf = lgb.train(
    params_lgb, # 使用贝叶斯调参最优参数
    lgb_train, 
    num_boost_round=20,
    valid_sets = [lgb_val], 
    verbose_eval=5, 
    init_model = clf,
    early_stopping_rounds=5,
    callbacks=[lgb.reset_parameter(learning_rate=[0.1] * 10 + [0.01] * 10)])
val_pre = clf.predict(test, num_iteration=clf.best_iteration)
print(roc_auc_score(y_test, val_pre))